In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.6 MB/s eta 0:00:00


In [8]:
csv_path = '/content/drive/My Drive/data/character_traits_scores.csv'
df = pd.read_csv(csv_path)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1696 entries, 0 to 1695
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Unnamed: 0              1696 non-null   object
 1   Unnamed: 1              1696 non-null   object
 2   Leadership              1696 non-null   int64 
 3   Communication           1696 non-null   int64 
 4   Teamwork                1696 non-null   int64 
 5   Problem Solving         1696 non-null   int64 
 6   Creativity              1696 non-null   int64 
 7   Adaptability            1696 non-null   int64 
 8   Work Ethic              1696 non-null   int64 
 9   Time Management         1696 non-null   int64 
 10  Interpersonal Skills    1696 non-null   int64 
 11  Attention to Detail     1696 non-null   int64 
 12  Initiative              1696 non-null   int64 
 13  Analytical Thinking     1696 non-null   int64 
 14  Emotional Intelligence  1696 non-null   int64 
 15  Inte

In [11]:
df.head()

,Unnamed: 0,Unnamed: 1,Leadership,Communication,Teamwork,Problem Solving,Creativity,Adaptability,Work Ethic,Time Management,...,Analytical Thinking,Emotional Intelligence,Integrity,Resilience,Cultural Awareness,Years of Experience,Programming Languages,Technical Skills,Office Tools,Job Description
0,DIGITAL-MEDIA,DIRECTOR,0,0,3,3,0,1,2,0,...,0,0,0,0,0,0,0,2,1,Behavioral Targeting Comscore SimilarWeb Kensh...
1,DIGITAL-MEDIA,MEDIA ACTIVITIES SPECIALIST,2,0,0,3,0,0,2,3,...,1,0,0,2,1,0,0,0,0,Summary Multi-Tasking Media Relations Results-...
2,DIGITAL-MEDIA,"SENIOR MANAGER, DIGITAL MARKETING",3,0,0,3,0,0,2,3,...,0,0,0,0,1,0,0,0,0,", State Created Customer Lifecycle Program gen..."
3,DIGITAL-MEDIA,DIGITAL PROJECT MANAGER,3,0,1,3,1,1,1,3,...,2,0,0,1,1,0,0,1,0,Created metrics to evaluate: Who is visiting 6...
4,DIGITAL-MEDIA,MONITOR TECH,2,3,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,2,Ability to perform duties under critical deadl...


In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import PyPDF2
import re
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from google.colab import files

# טעינת הנתונים
csv_path = '/content/drive/My Drive/data/character_traits_scores.csv'
df = pd.read_csv(csv_path)

# הצגת שמות העמודות
print("Column names in the dataset:")
print(df.columns)

# הכנת הנתונים למודל
X = df.iloc[:, 2:22]  # תכונות האופי
trait_names = X.columns.tolist()
y = df['Unnamed: 1']  # מקצועות
domains = df['Unnamed: 0']  # תחומים

# נרמול הנתונים
scaler = MinMaxScaler(feature_range=(0, 3))
X_scaled = scaler.fit_transform(X)

# המרת המקצועות לקטגוריות
y = pd.get_dummies(y)

# בניית המודל של רשת הנוירונים
model = Sequential()
model.add(Dense(128, input_dim=X_scaled.shape[1], activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))

# קומפילציה של המודל
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# חלוקת הנתונים לסט אימון וסט בדיקה
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# אימון המודל
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# מילון מורחב של מילים וביטויים קשורים לכל תכונה עם משקולות
trait_related_words = {
    "Leadership": [("lead", 1), ("manage", 1), ("direct", 1), ("charge nurse", 2), ("assigned and delegated", 2), ("coordinated", 1)],
    "Communication": [("communicate", 1), ("report", 1), ("consult", 1), ("coordinate", 1), ("collaborate", 1), ("therapeutic relationship", 2)],
    "Teamwork": [("team", 1), ("collaborate", 1), ("coordinate", 1), ("multi-disciplinary", 2), ("healthcare team", 2)],
    "Problem Solving": [("resolve", 1), ("handle", 1), ("emergency", 1), ("de-escalating", 2), ("interventions", 1)],
    "Creativity": [("innovative", 2), ("develop", 1), ("create", 1), ("implement", 1)],
    "Adaptability": [("adapt", 1), ("flexible", 1), ("various units", 2), ("different patients", 2), ("diverse experience", 2)],
    "Work Ethic": [("dedicated", 1), ("committed", 1), ("strong work ethic", 2), ("efficiently", 1)],
    "Time Management": [("prioritize", 2), ("manage time", 2), ("fast paced", 1)],
    "Interpersonal Skills": [("empathetic", 1), ("compassionate", 1), ("therapeutic relationship", 2), ("patient interaction", 1)],
    "Attention to Detail": [("detail oriented", 2), ("accurate", 1), ("precise", 1), ("monitored", 1)],
    "Initiative": [("proactive", 2), ("initiated", 1), ("implemented", 1)],
    "Analytical Thinking": [("assess", 1), ("evaluate", 1), ("analyze", 1), ("critical thinking", 2)],
    "Emotional Intelligence": [("empathy", 2), ("emotional support", 2), ("de-escalating", 1)],
    "Integrity": [("ethical", 2), ("professional", 1), ("in accordance with policy", 2)],
    "Resilience": [("handle stress", 2), ("busy unit", 1), ("challenging patients", 2)],
    "Cultural Awareness": [("diverse", 1), ("cultural", 1), ("multicultural", 2)],
    "Programming Languages": [("computer skills", 1), ("electronic health records", 2)],
    "Technical Skills": [("medical procedures", 1), ("administering medication", 1), ("vital signs", 1)],
    "Office Tools": [("documentation", 1), ("record keeping", 1), ("reports", 1)]
}

def extract_features(text):
    features = {trait: 0 for trait in trait_names}  # Initialize all traits to 0
    for trait, keywords in trait_related_words.items():
        if trait in trait_names:  # Ensure we only process traits that are in our original dataset
            score = sum([weight * len(re.findall(r'\b' + re.escape(keyword.lower()) + r'\b', text.lower())) for keyword, weight in keywords])
            features[trait] = min(score, 3)  # מגביל את הציון ל-3

    # חילוץ שנות ניסיון
    if 'Years of Experience' in trait_names:
        experience_matches = re.findall(r'(\d+)\s*years?\s*(of)?\s*experience', text, re.IGNORECASE)
        if experience_matches:
            total_experience = sum(int(match[0]) for match in experience_matches)
            features['Years of Experience'] = total_experience
        else:
            features['Years of Experience'] = 0

    return pd.DataFrame([features])

def process_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

def analyze_resume(file_path):
    text = process_pdf(file_path)
    features = extract_features(text)
    print("Extracted features:")
    print(features)

    # וידוא שכל העמודות נמצאות ובסדר הנכון
    for trait in trait_names:
        if trait not in features.columns:
            features[trait] = 0
    features = features[trait_names]

    # נרמול התכונות
    scaled_features = scaler.transform(features)
    scaled_features_df = pd.DataFrame(scaled_features, columns=trait_names)

    predictions = model.predict(scaled_features_df)
    predicted_professions = pd.DataFrame(predictions, columns=y.columns)

    # החלפת הערך המנובא של 'Years of Experience' בערך האמיתי אם קיים
    if 'Years of Experience' in trait_names:
        predicted_professions['Years of Experience'] = features['Years of Experience'].values[0]

    top_5_professions = predicted_professions.T.sort_values(by=0, ascending=False).head(5)

    return top_5_professions

# העלאת קובץ PDF
uploaded = files.upload()
file_path = next(iter(uploaded))

# ניתוח קורות החיים
results = analyze_resume(file_path)
print("\nTop 5 professions with their corresponding fields:")

# הצגת התוצאות
top_professions = results.index.tolist()
for profession in top_professions:
    matching_rows = df[df['Unnamed: 1'] == profession]
    if not matching_rows.empty:
        domain = matching_rows['Unnamed: 0'].values[0]
        print(f"{profession}: {domain}")
    else:
        print(f"{profession}: Unknown domain")


Column names in the dataset:
Index(['Unnamed: 0', 'Unnamed: 1', 'Leadership', 'Communication', 'Teamwork',
       'Problem Solving', 'Creativity', 'Adaptability', 'Work Ethic',
       'Time Management', 'Interpersonal Skills', 'Attention to Detail',
       'Initiative', 'Analytical Thinking', 'Emotional Intelligence',
       'Integrity', 'Resilience', 'Cultural Awareness', 'Years of Experience',
       'Programming Languages', 'Technical Skills', 'Office Tools',
       'Job Description'],
      dtype='object')
Epoch 1/50
43/43 [==============================] - 2s 11ms/step - loss: 7.4434 - accuracy: 0.0000e+00 - val_loss: 7.4529 - val_accuracy: 0.0000e+00
Epoch 2/50
43/43 [==============================] - 0s 7ms/step - loss: 7.3171 - accuracy: 0.0029 - val_loss: 7.6200 - val_accuracy: 0.0029
Epoch 3/50
43/43 [==============================] - 0s 7ms/step - loss: 7.1294 - accuracy: 0.0044 - val_loss: 8.3136 - val_accuracy: 0.0059
Epoch 4/50
43/43 [==============================] - 0s 

Saving 75435017.pdf to 75435017 (2).pdf
Extracted features:
   Leadership  Communication  Teamwork  Problem Solving  Creativity  \
0           3              3         3                3           2   

   Adaptability  Work Ethic  Time Management  Interpersonal Skills  \
0             0           3                1                     3   

   Attention to Detail  Initiative  Analytical Thinking  \
0                    3           0                    3   

   Emotional Intelligence  Integrity  Resilience  Cultural Awareness  \
0                       1          1           0                   0   

   Years of Experience  Programming Languages  Technical Skills  Office Tools  
0                   10                      0                 3             0  
1/1 [==============================] - 0s 135ms/step

Top 5 professions with their corresponding fields:
Years of Experience: Unknown domain
MANAGER: PUBLIC-RELATIONS
BUSINESS SYSTEMS ANALYST I: INFORMATION-TECHNOLOGY
VICE PRESIDENT

# Chapter 1: Loading the data and displaying column names

In [14]:
import pandas as pd

# טעינת הנתונים
csv_path = '/content/drive/My Drive/data/character_traits_scores.csv'
df = pd.read_csv(csv_path)

# הצגת שמות העמודות
print("Column names in the dataset:")
print(df.columns)


Column names in the dataset:
Index(['Unnamed: 0', 'Unnamed: 1', 'Leadership', 'Communication', 'Teamwork',
       'Problem Solving', 'Creativity', 'Adaptability', 'Work Ethic',
       'Time Management', 'Interpersonal Skills', 'Attention to Detail',
       'Initiative', 'Analytical Thinking', 'Emotional Intelligence',
       'Integrity', 'Resilience', 'Cultural Awareness', 'Years of Experience',
       'Programming Languages', 'Technical Skills', 'Office Tools',
       'Job Description'],
      dtype='object')


# Step 2: preparing the data for the model

In [15]:
from sklearn.preprocessing import MinMaxScaler

# הכנת הנתונים למודל
X = df.iloc[:, 2:22]  # תכונות האופי
trait_names = X.columns.tolist()
y = df['Unnamed: 1']  # מקצועות
domains = df['Unnamed: 0']  # תחומים

# נרמול הנתונים
scaler = MinMaxScaler(feature_range=(0, 3))
X_scaled = scaler.fit_transform(X)

# המרת המקצועות לקטגוריות
y = pd.get_dummies(y)


# Cell 3: Building and training the model

In [19]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# בניית המודל של רשת הנוירונים
model = Sequential()
model.add(Dense(256, input_dim=X_scaled.shape[1], activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))

# קומפילציה של המודל
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# חלוקת הנתונים לסט אימון וסט בדיקה
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# אימון המודל
model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/1000
43/43 [==============================] - 2s 13ms/step - loss: 7.4185 - accuracy: 0.0000e+00 - val_loss: 7.4098 - val_accuracy: 0.0029
Epoch 2/1000
43/43 [==============================] - 0s 7ms/step - loss: 7.3633 - accuracy: 0.0015 - val_loss: 7.4837 - val_accuracy: 0.0000e+00
Epoch 3/1000
43/43 [==============================] - 0s 8ms/step - loss: 7.3091 - accuracy: 0.0044 - val_loss: 7.8345 - val_accuracy: 0.0000e+00
Epoch 4/1000
43/43 [==============================] - 0s 8ms/step - loss: 7.1633 - accuracy: 0.0029 - val_loss: 8.1369 - val_accuracy: 0.0029
Epoch 5/1000
43/43 [==============================] - 0s 8ms/step - loss: 6.9430 - accuracy: 0.0037 - val_loss: 8.9195 - val_accuracy: 0.0000e+00
Epoch 6/1000
43/43 [==============================] - 0s 7ms/step - loss: 6.6824 - accuracy: 0.0044 - val_loss: 9.6145 - val_accuracy: 0.0000e+00
Epoch 7/1000
43/43 [==============================] - 0s 8ms/step - loss: 6.2848 - accuracy: 0.0170 - val_loss: 10.7284 - val_a

# Cell 4: Definition of the dictionary of words and phrases related to each feature

In [20]:
import re

# מילון מורחב של מילים וביטויים קשורים לכל תכונה עם משקולות
trait_related_words = {
    "Leadership": [("lead", 1), ("manage", 1), ("direct", 1), ("charge nurse", 2), ("assigned and delegated", 2), ("coordinated", 1)],
    "Communication": [("communicate", 1), ("report", 1), ("consult", 1), ("coordinate", 1), ("collaborate", 1), ("therapeutic relationship", 2)],
    "Teamwork": [("team", 1), ("collaborate", 1), ("coordinate", 1), ("multi-disciplinary", 2), ("healthcare team", 2)],
    "Problem Solving": [("resolve", 1), ("handle", 1), ("emergency", 1), ("de-escalating", 2), ("interventions", 1)],
    "Creativity": [("innovative", 2), ("develop", 1), ("create", 1), ("implement", 1)],
    "Adaptability": [("adapt", 1), ("flexible", 1), ("various units", 2), ("different patients", 2), ("diverse experience", 2)],
    "Work Ethic": [("dedicated", 1), ("committed", 1), ("strong work ethic", 2), ("efficiently", 1)],
    "Time Management": [("prioritize", 2), ("manage time", 2), ("fast paced", 1)],
    "Interpersonal Skills": [("empathetic", 1), ("compassionate", 1), ("therapeutic relationship", 2), ("patient interaction", 1)],
    "Attention to Detail": [("detail oriented", 2), ("accurate", 1), ("precise", 1), ("monitored", 1)],
    "Initiative": [("proactive", 2), ("initiated", 1), ("implemented", 1)],
    "Analytical Thinking": [("assess", 1), ("evaluate", 1), ("analyze", 1), ("critical thinking", 2)],
    "Emotional Intelligence": [("empathy", 2), ("emotional support", 2), ("de-escalating", 1)],
    "Integrity": [("ethical", 2), ("professional", 1), ("in accordance with policy", 2)],
    "Resilience": [("handle stress", 2), ("busy unit", 1), ("challenging patients", 2)],
    "Cultural Awareness": [("diverse", 1), ("cultural", 1), ("multicultural", 2)],
    "Programming Languages": [("computer skills", 1), ("electronic health records", 2)],
    "Technical Skills": [("medical procedures", 1), ("administering medication", 1), ("vital signs", 1)],
    "Office Tools": [("documentation", 1), ("record keeping", 1), ("reports", 1)]
}


# Cell 5: Defining functions for processing the PDF and extracting the features

In [21]:
import PyPDF2

def extract_features(text):
    features = {trait: 0 for trait in trait_names}  # Initialize all traits to 0
    for trait, keywords in trait_related_words.items():
        if trait in trait_names:  # Ensure we only process traits that are in our original dataset
            score = sum([weight * len(re.findall(r'\b' + re.escape(keyword.lower()) + r'\b', text.lower())) for keyword, weight in keywords])
            features[trait] = min(score, 3)  # מגביל את הציון ל-3

    # חילוץ שנות ניסיון
    if 'Years of Experience' in trait_names:
        experience_matches = re.findall(r'(\d+)\s*years?\s*(of)?\s*experience', text, re.IGNORECASE)
        if experience_matches:
            total_experience = sum(int(match[0]) for match in experience_matches)
            features['Years of Experience'] = total_experience
        else:
            features['Years of Experience'] = 0

    return pd.DataFrame([features])

def process_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text


# Cell 6: CV analysis and career prediction

In [23]:
from google.colab import files

def analyze_resume(file_path):
    text = process_pdf(file_path)
    features = extract_features(text)
    print("Extracted features:")
    print(features)

    # וידוא שכל העמודות נמצאות ובסדר הנכון
    for trait in trait_names:
        if trait not in features.columns:
            features[trait] = 0
    features = features[trait_names]

    # נרמול התכונות
    scaled_features = scaler.transform(features)
    scaled_features_df = pd.DataFrame(scaled_features, columns=trait_names)

    predictions = model.predict(scaled_features_df)
    predicted_professions = pd.DataFrame(predictions, columns=y.columns)

    # החלפת הערך המנובא של 'Years of Experience' בערך האמיתי אם קיים
    if 'Years of Experience' in trait_names:
        predicted_professions['Years of Experience'] = features['Years of Experience'].values[0]

    top_5_professions = predicted_professions.T.sort_values(by=0, ascending=False).head(5)

    return top_5_professions

# העלאת קובץ PDF
uploaded = files.upload()
file_path = next(iter(uploaded))

# ניתוח קורות החיים
results = analyze_resume(file_path)
print("\nTop 5 professions with their corresponding fields:")

# הצגת התוצאות
top_professions = results.index.tolist()
for profession in top_professions:
    matching_rows = df[df['Unnamed: 1'] == profession]
    if not matching_rows.empty:
        domain = matching_rows['Unnamed: 0'].values[0]
        print(f"{profession}: {domain}")
    else:
        print(f"{profession}: Unknown domain")


Saving 82649935.pdf to 82649935.pdf
Extracted features:
   Leadership  Communication  Teamwork  Problem Solving  Creativity  \
0           3              3         1                0           1   

   Adaptability  Work Ethic  Time Management  Interpersonal Skills  \
0             0           0                0                     0   

   Attention to Detail  Initiative  Analytical Thinking  \
0                    1           0                    1   

   Emotional Intelligence  Integrity  Resilience  Cultural Awareness  \
0                       0          1           0                   0   

   Years of Experience  Programming Languages  Technical Skills  Office Tools  
0                    0                      1                 0             3  
1/1 [==============================] - 0s 26ms/step

Top 5 professions with their corresponding fields:
ASSISTANT MANAGER/ SALES AND RELOCATION SPECIALIST: CONSTRUCTION
LIBRARY AIDE ( FULL-TIME): ARTS
INFORMATION TECHNOLOGY (INTERNSHIP)

# SAVE TH MODEL / SCALER

In [24]:
# שמירת המודל המאומן
model.save('/content/drive/My Drive/character_traits_model.h5')

# שמירת הסקלר המאומן
import joblib
joblib.dump(scaler, '/content/drive/My Drive/character_traits_scaler.pkl')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


['/content/drive/My Drive/character_traits_scaler.pkl']